In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/MSCOCO/Flicker8k_Dataset.zip" -d "/content"

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers
import pickle

# ---------------- HYPERPARAMETERS ----------------
IMG_SIZE = 224
PATCH_SIZE = 16
NUM_PATCHES = (IMG_SIZE // PATCH_SIZE) ** 2

EMBED_DIM = 256
NUM_HEADS = 4
MLP_DIM = 512

ENCODER_LAYERS = 6
DECODER_LAYERS = 6

MAX_LEN = 40
BATCH_SIZE = 8
EPOCHS_STAGE1 = 5 #20
EPOCHS_STAGE2 = 4 #15

DROPOUT = 0.1

# ============================================================================
# 180D MSCOCO FEATURE DIMENSIONS
# ============================================================================
# Complete MSCOCO features:
# - 80D: Object detection (person, car, dog, etc.)
# - 91D: Stuff detection (sky, grass, water, beach, etc.)
# - 9D: Scene statistics
MSCOCO_OBJECTS_DIM = 80
MSCOCO_STUFF_DIM = 91
SCENE_STATS_DIM = 9

# Total context dimension
CONTEXT_DIM = MSCOCO_OBJECTS_DIM + MSCOCO_STUFF_DIM + SCENE_STATS_DIM  # 180


print("MODEL ARCHITECTURE - 180D MSCOCO FEATURES")

print(f"Input 1: Image ({IMG_SIZE}x{IMG_SIZE}x3)")
print(f"Input 2: Context vector ({CONTEXT_DIM}D)")
print(f"  - Objects: {MSCOCO_OBJECTS_DIM}D (person, car, dog, ...)")
print(f"  - Stuff: {MSCOCO_STUFF_DIM}D (sky, grass, water, beach, ...)")
print(f"  - Scene stats: {SCENE_STATS_DIM}D")

# ---------------- PATHS ----------------
CAPTION_FILE = "/content/Flickr8k_text/Flickr8k.token.txt"
IMG_DIR = "/content/Flicker8k_Dataset/"

# NEW: 180D MSCOCO features CSV
MSCOCO_FEATURES_FILE = "/content/drive/MyDrive/MSCOCO/trainingdata/mscoco_object_stuff_detection.csv"

CHECKPOINT_DIR = "/content/drive/MyDrive/MSCOCO/trainingdata/checkpoints"
STAGE1_CHECKPOINT = os.path.join(CHECKPOINT_DIR, "stage1_latest.weights.h5")
STAGE2_CHECKPOINT = os.path.join(CHECKPOINT_DIR, "stage2_latest.weights.h5")
EPOCH_TRACKER = os.path.join(CHECKPOINT_DIR, "training_progress.pkl")

os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# ============================================================================
# CHECKPOINT MANAGEMENT
# ============================================================================

def save_training_progress(stage, epoch, history_dict):
    progress = {
        'stage': stage,
        'completed_epochs': epoch,
        'history': history_dict
    }
    with open(EPOCH_TRACKER, 'wb') as f:
        pickle.dump(progress, f)
    print(f"\n Saved progress: Stage {stage}, Epoch {epoch}")

def load_training_progress():
    if os.path.exists(EPOCH_TRACKER):
        with open(EPOCH_TRACKER, 'rb') as f:
            progress = pickle.load(f)
        print(f"\n Found previous training:")
        print(f"  Stage: {progress['stage']}")
        print(f"  Completed epochs: {progress['completed_epochs']}")
        return progress
    return None

def get_latest_checkpoint(stage):
    if stage == 1:
        if os.path.exists(STAGE1_CHECKPOINT):
            return STAGE1_CHECKPOINT
    elif stage == 2:
        if os.path.exists(STAGE2_CHECKPOINT):
            return STAGE2_CHECKPOINT
    return None

# ---------------- LOAD DATA ----------------
print("LOADING DATA")

# Load captions
df = pd.read_csv(CAPTION_FILE, sep='\t', header=None, names=["image_id", "caption"])
df["image_id"] = df["image_id"].apply(lambda x: x.split("#")[0])
df = df.rename(columns={"image_id": "filename"})
print(f"Loaded {len(df)} captions")

# ============================================================================
# LOAD 180D MSCOCO FEATURES
# ============================================================================

print("LOADING 180D MSCOCO FEATURES")

if not os.path.exists(MSCOCO_FEATURES_FILE):
    print(f" ERROR: MSCOCO features file not found!")
    print(f"   Expected: {MSCOCO_FEATURES_FILE}")
    print("\nYou need to run the feature extraction script first:")
    print("   python mscoco_complete_extractor.py")
    print("\nThis will generate the 180D features CSV file.")
    exit(1)

print(f"Loading from: {MSCOCO_FEATURES_FILE}")
mscoco_df = pd.read_csv(MSCOCO_FEATURES_FILE)

print(f"  Loaded MSCOCO features")
print(f"  Rows: {len(mscoco_df)}")
print(f"  Columns: {len(mscoco_df.columns)}")

# Expected format: filename, feat_0, feat_1, ..., feat_179
# Total: 181 columns (1 filename + 180 features)

if len(mscoco_df.columns) < 181:
    print(f"\n WARNING: Expected 181 columns (1 filename + 180 features)")
    print(f"   Found: {len(mscoco_df.columns)} columns")
    print(f"   Will pad with zeros if needed")

# Extract features into dictionary
mscoco_data = {}

for _, row in mscoco_df.iterrows():
    filename = row[0] if isinstance(row[0], str) else row['filename']

    # Extract 180 features (columns 1-180)
    if len(row) >= 181:
        features_180d = row[1:181].values.astype(np.float32).tolist()
    else:
        # Pad if needed
        available_features = row[1:].values.astype(np.float32).tolist()
        padding = [0.0] * (CONTEXT_DIM - len(available_features))
        features_180d = available_features + padding

    mscoco_data[filename] = features_180d

print(f"Processed {len(mscoco_data)} images")

# Sample verification
sample_file = list(mscoco_data.keys())[0]
sample_features = mscoco_data[sample_file]
print(f"\nSample verification:")
print(f"  Filename: {sample_file}")
print(f"  Feature vector length: {len(sample_features)}")
print(f"  Non-zero features: {np.count_nonzero(sample_features)}")
print(f"  Feature range: [{min(sample_features):.3f}, {max(sample_features):.3f}]")

# ============================================================================
# CREATE CONTEXT VECTORS
# ============================================================================

print("CREATING CONTEXT VECTORS")

def create_context_vector(filename):
    """
    Get 180D MSCOCO features as context vector

    Returns: 180D vector
      - [0:80] = Object features (person, car, dog, ...)
      - [80:171] = Stuff features (sky, grass, water, beach, ...)
      - [171:180] = Scene statistics
    """
    return mscoco_data.get(filename, [0.0] * CONTEXT_DIM)

# Add context vector to dataframe
df["context"] = df["filename"].apply(create_context_vector)

# Verify context vectors
print(f"Context vectors created: {len(df)}")
print(f"Context dimension: {len(df.iloc[0]['context'])}")

# Filter complete data
initial_count = len(df)
df = df[df["context"].apply(lambda x: len(x) == CONTEXT_DIM)]
final_count = len(df)

print(f"\nData filtering:")
print(f"  Initial captions: {initial_count}")
print(f"  Complete data: {final_count}")
print(f"  Filtered out: {initial_count - final_count}")

if final_count == 0:
    print("\n ERROR: No valid data found!")
    print("   Check that filenames match between caption file and MSCOCO features CSV")
    exit(1)

# Text preprocessing
print("\n" + "="*80)
print("PREPROCESSING CAPTIONS")
print("="*80)

def clean_caption(c):
    c = c.lower()
    c = re.sub(r"[^a-z0-9 ]", "", c)
    return "startseq " + c.strip() + " endseq"

df["caption"] = df["caption"].apply(clean_caption)

tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(df["caption"].values)

word_index = tokenizer.word_index
VOCAB_SIZE = len(word_index) + 1

seqs = tokenizer.texts_to_sequences(df["caption"])
seqs = tf.keras.preprocessing.sequence.pad_sequences(seqs, maxlen=MAX_LEN, padding="post")
df["seq"] = seqs.tolist()

print(f"Vocabulary size: {VOCAB_SIZE}")
print(f"Caption sequences created")

# Sample caption
sample_idx = 0
print(f"\nSample caption:")
print(f"  Original: {df.iloc[sample_idx]['caption']}")
print(f"  Sequence length: {len(df.iloc[sample_idx]['seq'])}")

# Image loader
def load_image(path):
    try:
        img = Image.open(path).convert("RGB")
        img = img.resize((IMG_SIZE, IMG_SIZE))
        arr = np.array(img).astype(np.float32) / 255.0
        return arr
    except Exception as e:
        return None

# ============================================================================
# DATA GENERATOR - 2 inputs + sequence
# ============================================================================

def data_generator():
    """Yields ((image, context), seq_target)"""
    for _, row in df.iterrows():
        filename = row["filename"]
        img_path = os.path.join(IMG_DIR, filename)

        if not os.path.exists(img_path) and img_path.endswith(".jpg.1"):
            candidate = img_path.replace(".jpg.1", ".jpg")
            if os.path.exists(candidate):
                img_path = candidate

        if not os.path.exists(img_path):
            continue

        img = load_image(img_path)
        if img is None:
            continue

        seq = np.array(row["seq"], dtype=np.int32)
        inp = seq[:-1]
        out = seq[1:]

        if inp.shape[0] != MAX_LEN - 1 or out.shape[0] != MAX_LEN - 1:
            continue

        context = np.array(row["context"], dtype=np.float32)

        yield (img, context, inp), out

output_signature = (
    (tf.TensorSpec(shape=(IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
     tf.TensorSpec(shape=(CONTEXT_DIM,), dtype=tf.float32),
     tf.TensorSpec(shape=(MAX_LEN-1,), dtype=tf.int32)),
    tf.TensorSpec(shape=(MAX_LEN-1,), dtype=tf.int32)
)

dataset = tf.data.Dataset.from_generator(data_generator, output_signature=output_signature)
dataset = dataset.shuffle(2048).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("\n Dataset pipeline created")

# ---------------- MODEL COMPONENTS ----------------

class PatchExtract(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [tf.shape(images)[0], -1, patch_dims])
        return patches

class PatchEmbedding(layers.Layer):
    def __init__(self, num_patches, embed_dim):
        super().__init__()
        self.proj = layers.Dense(embed_dim)
        self.pos = layers.Embedding(num_patches, embed_dim)

    def call(self, patches):
        pos_ids = tf.range(start=0, limit=NUM_PATCHES, delta=1)
        pos_emb = self.pos(pos_ids)
        pos_emb = tf.expand_dims(pos_emb, axis=0)
        x = self.proj(patches) + pos_emb
        return x

class EncoderBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_dim):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads)
        self.ln1 = layers.LayerNormalization(epsilon=1e-6)
        self.ff = tf.keras.Sequential([
            layers.Dense(mlp_dim, activation="gelu"),
            layers.Dense(embed_dim)
        ])
        self.ln2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, x):
        att_out = self.att(x, x)
        x = self.ln1(x + att_out)
        ff_out = self.ff(x)
        x = self.ln2(x + ff_out)
        return x

def build_vit_encoder():
    img = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    patches = PatchExtract(PATCH_SIZE)(img)
    x = PatchEmbedding(NUM_PATCHES, EMBED_DIM)(patches)

    for _ in range(ENCODER_LAYERS):
        x = EncoderBlock(EMBED_DIM, NUM_HEADS, MLP_DIM)(x)

    return tf.keras.Model(img, x, name="vit_encoder")

class ContextFusion(layers.Layer):
    """Fuses 180D MSCOCO context with visual features"""

    def __init__(self, embed_dim):
        super().__init__()
        self.context_proj = layers.Dense(embed_dim, name="context_projection")
        self.dropout = layers.Dropout(DROPOUT)
        self.ln = layers.LayerNormalization(epsilon=1e-6)

    def call(self, enc_out, context, training=False):
        """
        Args:
            enc_out: (batch, NUM_PATCHES, embed_dim) - visual features
            context: (batch, 180) - MSCOCO features (80 objects + 91 stuff + 9 stats)

        Returns:
            (batch, NUM_PATCHES + 1, embed_dim) - fused features
        """
        context_emb = self.context_proj(context)
        context_emb = self.dropout(context_emb, training=training)
        context_emb = tf.expand_dims(context_emb, 1)

        combined = tf.concat([enc_out, context_emb], axis=1)
        return self.ln(combined)

class DecoderBlock(layers.Layer):
    def __init__(self, embed_dim, ff_dim, num_heads):
        super().__init__()
        self.self_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads)
        self.cross_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads)
        self.ff = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dropout(DROPOUT),
            layers.Dense(embed_dim)
        ])
        self.ln1 = layers.LayerNormalization(epsilon=1e-6)
        self.ln2 = layers.LayerNormalization(epsilon=1e-6)
        self.ln3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(DROPOUT)
        self.dropout2 = layers.Dropout(DROPOUT)

    def _causal_mask(self, seq_len, batch_size):
        mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        mask = tf.cast(mask, tf.bool)
        mask = tf.reshape(mask, (1, seq_len, seq_len))
        return tf.repeat(mask, repeats=batch_size, axis=0)

    def call(self, x, enc_out, training=False):
        batch = tf.shape(x)[0]
        seq_len = tf.shape(x)[1]
        mask = self._causal_mask(seq_len, batch)

        att1 = self.self_att(x, x, attention_mask=mask)
        att1 = self.dropout1(att1, training=training)
        x = self.ln1(x + att1)

        att2 = self.cross_att(x, enc_out)
        att2 = self.dropout2(att2, training=training)
        x = self.ln2(x + att2)

        f = self.ff(x, training=training)
        x = self.ln3(x + f)
        return x

def build_decoder():
    seq_in = layers.Input(shape=(MAX_LEN-1,), dtype=tf.int32)
    enc_out = layers.Input(shape=(NUM_PATCHES + 1, EMBED_DIM))

    tok_emb = layers.Embedding(VOCAB_SIZE, EMBED_DIM)(seq_in)
    pos_ids = tf.range(start=0, limit=MAX_LEN-1, delta=1)
    pos_layer = layers.Embedding(MAX_LEN, EMBED_DIM)
    pos_emb = pos_layer(pos_ids)
    pos_emb = tf.expand_dims(pos_emb, 0)
    x = tok_emb + pos_emb

    for _ in range(DECODER_LAYERS):
        x = DecoderBlock(EMBED_DIM, MLP_DIM, NUM_HEADS)(x, enc_out)

    out = layers.Dense(VOCAB_SIZE)(x)
    return tf.keras.Model([seq_in, enc_out], out, name="decoder")

# ============================================================================
# BUILD MODEL
# ============================================================================

print("BUILDING MODEL")

encoder = build_vit_encoder()
decoder = build_decoder()
context_fusion = ContextFusion(EMBED_DIM)

img_inp = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="image_input")
context_inp = layers.Input(shape=(CONTEXT_DIM,), name="context_input")
seq_inp = layers.Input(shape=(MAX_LEN-1,), dtype=tf.int32, name="sequence_input")

enc_out = encoder(img_inp)
fused_out = context_fusion(enc_out, context_inp)
dec_out = decoder([seq_inp, fused_out])

model = tf.keras.Model([img_inp, context_inp, seq_inp], dec_out)

print(f"\n Model created")
print(f"  Encoder: {ENCODER_LAYERS} layers")
print(f"  Decoder: {DECODER_LAYERS} layers")
print(f"  Context: {CONTEXT_DIM}D (180D MSCOCO)")
print(f"  Vocabulary: {VOCAB_SIZE} words")

model.summary()

# ============================================================================
# CHECKPOINT SYSTEM
# ============================================================================

print("CHECKING FOR PREVIOUS TRAINING")

progress = load_training_progress()

if progress is None:
    print("No previous training found. Starting from scratch.")
    current_stage = 1
    stage1_start_epoch = 0
    stage2_start_epoch = 0
    training_history = {'stage1': {}, 'stage2': {}}
else:
    current_stage = progress['stage']
    completed_epochs = progress['completed_epochs']
    training_history = progress.get('history', {'stage1': {}, 'stage2': {}})

    if current_stage == 1:
        stage1_start_epoch = completed_epochs
        stage2_start_epoch = 0

        if stage1_start_epoch >= EPOCHS_STAGE1:
            current_stage = 2
            stage1_start_epoch = EPOCHS_STAGE1
            stage2_start_epoch = 0
        else:
            print(f"Resuming Stage 1 from epoch {stage1_start_epoch}/{EPOCHS_STAGE1}")
    else:
        stage1_start_epoch = EPOCHS_STAGE1
        stage2_start_epoch = completed_epochs

        if stage2_start_epoch >= EPOCHS_STAGE2:
            print(f"Training already complete!")
            exit(0)
        else:
            print(f"Resuming Stage 2 from epoch {stage2_start_epoch}/{EPOCHS_STAGE2}")

# ============================================================================
# STAGE 1: TRAIN DECODER ONLY
# ============================================================================

if current_stage == 1 and stage1_start_epoch < EPOCHS_STAGE1:
    print("\n" + "="*80)
    print(f"STAGE 1: DECODER TRAINING")
    print("="*80)

    encoder.trainable = False
    for layer in encoder.layers:
        layer.trainable = False

    enc_out = encoder(img_inp)
    fused_out = context_fusion(enc_out, context_inp)
    dec_out = decoder([seq_inp, fused_out])
    model = tf.keras.Model([img_inp, context_inp, seq_inp], dec_out)

    if stage1_start_epoch > 0:
        checkpoint = get_latest_checkpoint(1)
        if checkpoint:
            model.load_weights(checkpoint)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    class EpochCheckpoint(tf.keras.callbacks.Callback):
        def __init__(self, stage, start_epoch):
            super().__init__()
            self.stage = stage
            self.start_epoch = start_epoch

        def on_epoch_end(self, epoch, logs=None):
            actual_epoch = self.start_epoch + epoch + 1

            if self.stage == 1:
                self.model.save_weights(STAGE1_CHECKPOINT)
            else:
                self.model.save_weights(STAGE2_CHECKPOINT)

            save_training_progress(self.stage, actual_epoch, training_history)

    epoch_checkpoint = EpochCheckpoint(stage=1, start_epoch=stage1_start_epoch)
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-7)

    remaining_epochs = EPOCHS_STAGE1 - stage1_start_epoch

    history1 = model.fit(
        dataset,
        epochs=remaining_epochs,
        callbacks=[epoch_checkpoint, early_stop, reduce_lr],
        verbose=1
    )

    for key, values in history1.history.items():
        if key not in training_history['stage1']:
            training_history['stage1'][key] = []
        training_history['stage1'][key].extend(values)

    current_stage = 2
    stage2_start_epoch = 0

# ============================================================================
# STAGE 2: FINE-TUNE ENTIRE MODEL
# ============================================================================

if current_stage == 2 and stage2_start_epoch < EPOCHS_STAGE2:
    print("\n" + "="*80)
    print(f"STAGE 2: FINE-TUNING")
    print("="*80)

    encoder.trainable = True
    for layer in encoder.layers:
        layer.trainable = True

    enc_out = encoder(img_inp)
    fused_out = context_fusion(enc_out, context_inp)
    dec_out = decoder([seq_inp, fused_out])
    model = tf.keras.Model([img_inp, context_inp, seq_inp], dec_out)

    if stage2_start_epoch == 0:
        checkpoint = get_latest_checkpoint(1)
        if checkpoint:
            model.load_weights(checkpoint)
    else:
        checkpoint = get_latest_checkpoint(2)
        if checkpoint:
            model.load_weights(checkpoint)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    epoch_checkpoint = EpochCheckpoint(stage=2, start_epoch=stage2_start_epoch)
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-7)

    remaining_epochs = EPOCHS_STAGE2 - stage2_start_epoch

    history2 = model.fit(
        dataset,
        epochs=remaining_epochs,
        callbacks=[epoch_checkpoint, early_stop, reduce_lr],
        verbose=1
    )

    for key, values in history2.history.items():
        if key not in training_history['stage2']:
            training_history['stage2'][key] = []
        training_history['stage2'][key].extend(values)

# ============================================================================
# SAVE FINAL MODEL
# ============================================================================

print("SAVING FINAL MODEL")

final_weights = "/content/drive/MyDrive/MSCOCO/trainingdata/final_model.weights.h5"
model.save_weights(final_weights)

tokenizer_path = "/content/drive/MyDrive/MSCOCO/trainingdata/tokenizer.pkl"
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)

config = {
    'IMG_SIZE': IMG_SIZE,
    'PATCH_SIZE': PATCH_SIZE,
    'NUM_PATCHES': NUM_PATCHES,
    'EMBED_DIM': EMBED_DIM,
    'NUM_HEADS': NUM_HEADS,
    'MLP_DIM': MLP_DIM,
    'ENCODER_LAYERS': ENCODER_LAYERS,
    'DECODER_LAYERS': DECODER_LAYERS,
    'MAX_LEN': MAX_LEN,
    'CONTEXT_DIM': CONTEXT_DIM,
    'MSCOCO_OBJECTS_DIM': MSCOCO_OBJECTS_DIM,
    'MSCOCO_STUFF_DIM': MSCOCO_STUFF_DIM,
    'SCENE_STATS_DIM': SCENE_STATS_DIM,
    'VOCAB_SIZE': VOCAB_SIZE
}

config_path = "/content/drive/MyDrive/MSCOCO/trainingdata/model_config.pkl"
with open(config_path, 'wb') as f:
    pickle.dump(config, f)

history_path = "/content/drive/MyDrive/MSCOCO/trainingdata/training_history.pkl"
with open(history_path, 'wb') as f:
    pickle.dump(training_history, f)

print(f"Model weights: {final_weights}")
print(f"Tokenizer: {tokenizer_path}")
print(f"Configuration: {config_path}")
print(f"Training history: {history_path}")

print("\n" + "="*80)
print("TRAINING COMPLETE!")
print("="*80)
print(f"Architecture: Image + 180D MSCOCO Context")
print(f"  • 80D: Objects (person, car, dog, ...)")
print(f"  • 91D: Stuff (sky, grass, water, beach, ...)")
print(f"  • 9D: Scene statistics")
print(f"Total epochs: {EPOCHS_STAGE1 + EPOCHS_STAGE2}")
print("="*80)

MODEL ARCHITECTURE - 180D MSCOCO FEATURES
Input 1: Image (224x224x3)
Input 2: Context vector (180D)
  - Objects: 80D (person, car, dog, ...)
  - Stuff: 91D (sky, grass, water, beach, ...)
  - Scene stats: 9D
LOADING DATA
Loaded 40460 captions
LOADING 180D MSCOCO FEATURES
Loading from: /content/drive/MyDrive/MSCOCO/trainingdata/mscoco_object_stuff_detection.csv
  Loaded MSCOCO features
  Rows: 8091
  Columns: 181


/tmp/ipython-input-757765191.py:141: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  filename = row[0] if isinstance(row[0], str) else row['filename']


Processed 8091 images

Sample verification:
  Filename: 2306674172_dc07c7f847.jpg
  Feature vector length: 180
  Non-zero features: 24
  Feature range: [0.000, 9.000]
CREATING CONTEXT VECTORS
Context vectors created: 40460
Context dimension: 180

Data filtering:
  Initial captions: 40460
  Complete data: 40460
  Filtered out: 0

PREPROCESSING CAPTIONS
Vocabulary size: 8832
Caption sequences created

Sample caption:
  Original: startseq a child in a pink dress is climbing up a set of stairs in an entry way endseq
  Sequence length: 40

 Dataset pipeline created
BUILDING MODEL

 Model created
  Encoder: 6 layers
  Decoder: 6 layers
  Context: 180D (180D MSCOCO)
  Vocabulary: 8832 words


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vit_encoder         │ (None, 196, 256)  │  3,409,664 │ image_input[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ context_input       │ (None, 180)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequence_input      │ (None, 39)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ context_fusion      │ (None, 197, 256)  │     46,848 │ vit_encoder[0][0… │
│ (ContextFusion)     │                   │            │ context_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder             │ (None, 39, 8832)  │  9,275,520 │ sequence_input[0… │
│ (Functional)        │                   │            │ context_fusion[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,732,032 (48.57 MB)

 Trainable params: 12,732,032 (48.57 MB)

 Non-trainable params: 0 (0.00 B)

CHECKING FOR PREVIOUS TRAINING

 Found previous training:
  Stage: 2
  Completed epochs: 4
Training already complete!
SAVING FINAL MODEL
Model weights: /content/drive/MyDrive/MSCOCO/trainingdata/final_model.weights.h5
Tokenizer: /content/drive/MyDrive/MSCOCO/trainingdata/tokenizer.pkl
Configuration: /content/drive/MyDrive/MSCOCO/trainingdata/model_config.pkl
Training history: /content/drive/MyDrive/MSCOCO/trainingdata/training_history.pkl

TRAINING COMPLETE!
Architecture: Image + 180D MSCOCO Context
  • 80D: Objects (person, car, dog, ...)
  • 91D: Stuff (sky, grass, water, beach, ...)
  • 9D: Scene statistics
Total epochs: 9


In [ ]:
# Save the entire model as a `.keras` zip archive.
model.save('/content/drive/MyDrive/MSCOCO/trainingdata/final_model.keras')
model.save('/content/drive/MyDrive/MSCOCO/trainingdata/final_model.h5')

# INFERENCE

In [ ]:
"""
COMPLETE AUTOMATIC INFERENCE SCRIPT - 180D MSCOCO Features
Input: Image file
Output: Generated caption

Uses 180D MSCOCO features:
- 80D: Object detection (person, car, dog, etc.)
- 91D: Stuff detection (sky, grass, water, etc.)
- 9D: Scene statistics
"""

import os
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers
import pickle
import tensorflow_hub as hub

print("="*80)
print("AUTOMATIC IMAGE CAPTIONING - 180D MSCOCO Features")
print("Loading models and dependencies...")
print("="*80)

# ============================================================================
# MSCOCO CLASS DEFINITIONS
# ============================================================================

MSCOCO_OBJECTS = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
    'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat',
    'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
    'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop',
    'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
    'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

MSCOCO_STUFF = [
    'sky', 'grass', 'tree', 'mountain', 'hill', 'rock', 'water', 'sea', 'river', 'lake',
    'sand', 'snow', 'fog', 'clouds', 'bush', 'flower', 'leaves', 'branch', 'dirt', 'mud',
    'building', 'house', 'bridge', 'fence', 'wall', 'roof', 'door', 'window', 'stairs',
    'ceiling', 'floor', 'platform', 'pavement', 'road', 'railroad', 'ground',
    'cabinet', 'shelf', 'table', 'counter', 'carpet', 'rug', 'curtain', 'blanket',
    'pillow', 'towel', 'mirror', 'light', 'paper', 'cardboard', 'wood', 'metal',
    'plastic', 'glass', 'tile', 'brick', 'stone',
    'banner', 'net', 'tent', 'playingfield', 'fruit', 'vegetable', 'food', 'cloth',
    'textile', 'plant', 'gravel', 'moss', 'straw',
    # Pad to 91 classes
    'material', 'surface', 'landscape', 'scenery', 'background', 'foreground',
    'area', 'region', 'space', 'field', 'place', 'location', 'terrain', 'zone',
    'environment', 'setting', 'context', 'atmosphere', 'element', 'component',
    'structure', 'formation'
][:91]  # Ensure exactly 91

# ============================================================================
# LOAD MODEL CONFIGURATION
# ============================================================================

CONFIG_PATH = "/content/drive/MyDrive/MSCOCO/trainingdata/model_config.pkl"
WEIGHTS_PATH = "/content/drive/MyDrive/MSCOCO/trainingdata/final_model.weights.h5"
TOKENIZER_PATH = "/content/drive/MyDrive/MSCOCO/trainingdata/tokenizer.pkl"

print("\n[1/5] Loading configuration...")
with open(CONFIG_PATH, 'rb') as f:
    config = pickle.load(f)

IMG_SIZE = config['IMG_SIZE']
PATCH_SIZE = config['PATCH_SIZE']
NUM_PATCHES = config['NUM_PATCHES']
EMBED_DIM = config['EMBED_DIM']
NUM_HEADS = config['NUM_HEADS']
MLP_DIM = config['MLP_DIM']
ENCODER_LAYERS = config['ENCODER_LAYERS']
DECODER_LAYERS = config['DECODER_LAYERS']
MAX_LEN = config['MAX_LEN']
CONTEXT_DIM = config['CONTEXT_DIM']
VOCAB_SIZE = config['VOCAB_SIZE']

print(f"✓ Configuration loaded")
print(f"  Context dimension: {CONTEXT_DIM}D")

if CONTEXT_DIM != 180:
    print(f"\n  WARNING: Model expects {CONTEXT_DIM}D context, but 180D features will be extracted")
    print(f"  You need to retrain with 180D features if CONTEXT_DIM != 180")

# ============================================================================
# LOAD TOKENIZER
# ============================================================================

print("\n[2/5] Loading tokenizer...")
with open(TOKENIZER_PATH, 'rb') as f:
    tokenizer = pickle.load(f)

idx_to_word = {idx: word for word, idx in tokenizer.word_index.items()}
idx_to_word[0] = '<pad>'

print(f" Tokenizer loaded ({len(tokenizer.word_index)} words)")

# ============================================================================
# OBJECT DETECTOR (80 classes)
# ============================================================================

print("\n[3/5] Loading object detector...")

class ObjectDetector:
    """Detects MSCOCO Objects (80 classes)"""

    def __init__(self, confidence_threshold=0.3, max_objects=20):
        self.confidence_threshold = confidence_threshold
        self.max_objects = max_objects
        self.object_classes = MSCOCO_OBJECTS

        self.model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
        print("  ✓ Object detector loaded")

    def detect(self, image):
        """Detect objects and return 80D feature vector (class counts)"""
        if isinstance(image, str):
            image = cv2.imread(image)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Run detection
        input_tensor = tf.convert_to_tensor(image)[tf.newaxis, ...]
        detections = self.model(input_tensor)

        boxes = detections['detection_boxes'][0].numpy()
        classes = detections['detection_classes'][0].numpy().astype(int)
        scores = detections['detection_scores'][0].numpy()

        # Count objects by class
        class_counts = np.zeros(len(self.object_classes), dtype=np.float32)

        valid_indices = np.where(scores >= self.confidence_threshold)[0][:self.max_objects]

        detected_objects = []
        for idx in valid_indices:
            class_id = classes[idx] - 1  # COCO is 1-indexed
            if 0 <= class_id < len(self.object_classes):
                class_counts[class_id] += 1
                detected_objects.append({
                    'name': self.object_classes[class_id],
                    'confidence': float(scores[idx])
                })

        # Normalize by max count
        max_count = max(class_counts.max(), 1.0)
        class_counts = class_counts / max_count

        return class_counts, detected_objects

# ============================================================================
# STUFF DETECTOR (91 classes)
# ============================================================================

print("\n[4/5] Loading stuff detector...")

class StuffDetector:
    """Detects MSCOCO Stuff (91 classes) using heuristics"""

    def __init__(self, coverage_threshold=0.01):
        self.coverage_threshold = coverage_threshold
        self.stuff_classes = MSCOCO_STUFF
        print("  ✓ Stuff detector loaded (heuristic-based)")

    def detect(self, image):
        """Detect stuff and return 91D feature vector"""
        if isinstance(image, str):
            image = cv2.imread(image)

        h, w = image.shape[:2]

        # Convert to different color spaces
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        stuff_features = np.zeros(91, dtype=np.float32)

        # Top region for sky
        top_region = image[:h//4, :]
        blue_ratio = self._detect_color(top_region, 'blue')
        stuff_features[0] = min(blue_ratio * 2, 1.0)  # sky

        # Bottom region for grass
        bottom_region = image[h*3//4:, :]
        green_ratio = self._detect_color(bottom_region, 'green')
        stuff_features[1] = min(green_ratio * 2, 1.0)  # grass

        # Middle region for water
        middle_region = image[h//4:h*3//4, :]
        water_ratio = self._detect_color(middle_region, 'blue')
        stuff_features[6] = min(water_ratio * 1.5, 1.0)  # water

        # Sea (blue in lower half)
        sea_ratio = self._detect_color(bottom_region, 'blue')
        stuff_features[7] = min(sea_ratio * 1.8, 1.0)  # sea

        # Tree (green with texture)
        edges = cv2.Canny(gray, 50, 150)
        edge_density = np.sum(edges > 0) / (h * w)
        green_all = self._detect_color(image, 'green')
        stuff_features[2] = min(green_all * edge_density * 5, 1.0)  # tree

        # Building (gray, vertical structures)
        gray_ratio = self._detect_color(image, 'gray')
        stuff_features[20] = min(gray_ratio * 1.5, 1.0)  # building

        # Road (gray in bottom)
        road_ratio = self._detect_color(bottom_region, 'gray')
        stuff_features[33] = min(road_ratio * 1.5, 1.0)  # road

        # Wall
        wall_ratio = self._detect_color(middle_region, 'gray')
        stuff_features[24] = min(wall_ratio * 1.2, 1.0)  # wall

        # Ground (brown/tan)
        brown_ratio = self._detect_color(bottom_region, 'brown')
        stuff_features[35] = min(brown_ratio * 1.5, 1.0)  # ground

        # Sand (light brown/yellow in bottom)
        sand_ratio = self._detect_color(bottom_region, 'yellow')
        stuff_features[10] = min(sand_ratio * 1.2, 1.0)  # sand

        # Snow (white)
        white_ratio = self._detect_color(image, 'white')
        stuff_features[11] = min(white_ratio * 1.5, 1.0)  # snow

        # Mountain (gray/brown in top half)
        mountain_ratio = self._detect_color(top_region, 'gray')
        stuff_features[3] = min(mountain_ratio * 1.3, 1.0)  # mountain

        return stuff_features

    def _detect_color(self, region, color):
        """Detect color presence in image region"""
        hsv = cv2.cvtColor(region, cv2.COLOR_BGR2HSV)

        color_ranges = {
            'blue': ([100, 50, 50], [130, 255, 255]),
            'green': ([35, 40, 40], [85, 255, 255]),
            'gray': ([0, 0, 50], [180, 50, 200]),
            'brown': ([10, 50, 20], [30, 255, 200]),
            'yellow': ([20, 100, 100], [35, 255, 255]),
            'white': ([0, 0, 200], [180, 50, 255])
        }

        if color not in color_ranges:
            return 0.0

        lower, upper = color_ranges[color]
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        ratio = np.sum(mask > 0) / (region.shape[0] * region.shape[1])

        return ratio

# ============================================================================
# COMPLETE FEATURE EXTRACTOR (180D)
# ============================================================================

class CompleteFeatureExtractor:
    """
    Extracts 180D MSCOCO features:
    - 80D: Object counts
    - 91D: Stuff coverage
    - 9D: Scene statistics
    """

    def __init__(self):
        self.object_detector = ObjectDetector()
        self.stuff_detector = StuffDetector()
        print("\n✓ Complete feature extractor initialized (180D)")

    def extract_features(self, image):
        """Extract complete 180D feature vector"""
        # Objects (80D)
        object_features, detected_objects = self.object_detector.detect(image)

        # Stuff (91D)
        stuff_features = self.stuff_detector.detect(image)

        # Scene statistics (9D)
        scene_stats = self._compute_scene_stats(object_features, stuff_features)

        # Combine
        complete_features = np.concatenate([
            object_features,  # 80D
            stuff_features,   # 91D
            scene_stats       # 9D
        ])

        return complete_features.astype(np.float32), detected_objects

    def _compute_scene_stats(self, object_features, stuff_features):
        """Compute 9D scene statistics"""
        stats = [
            np.sum(object_features > 0),           # Number of object types
            np.sum(stuff_features > 0),            # Number of stuff types
            np.mean(object_features),              # Average object presence
            np.std(object_features),               # Object diversity
            np.mean(stuff_features),               # Average stuff coverage
            np.std(stuff_features),                # Stuff diversity
            np.max(object_features),               # Max object count
            np.max(stuff_features),                # Max stuff coverage
            (np.sum(object_features > 0) + np.sum(stuff_features > 0)) / 171
        ]
        return np.array(stats, dtype=np.float32)

# ============================================================================
# MODEL ARCHITECTURE
# ============================================================================

print("\n[5/5] Building model architecture...")

class PatchExtract(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [tf.shape(images)[0], -1, patch_dims])
        return patches

class PatchEmbedding(layers.Layer):
    def __init__(self, num_patches, embed_dim):
        super().__init__()
        self.proj = layers.Dense(embed_dim)
        self.pos = layers.Embedding(num_patches, embed_dim)

    def call(self, patches):
        pos_ids = tf.range(start=0, limit=NUM_PATCHES, delta=1)
        pos_emb = self.pos(pos_ids)
        pos_emb = tf.expand_dims(pos_emb, axis=0)
        x = self.proj(patches) + pos_emb
        return x

class EncoderBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_dim):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads)
        self.ln1 = layers.LayerNormalization(epsilon=1e-6)
        self.ff = tf.keras.Sequential([
            layers.Dense(mlp_dim, activation="gelu"),
            layers.Dense(embed_dim)
        ])
        self.ln2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, x):
        att_out = self.att(x, x)
        x = self.ln1(x + att_out)
        ff_out = self.ff(x)
        x = self.ln2(x + ff_out)
        return x

class ContextFusion(layers.Layer):
    def __init__(self, embed_dim):
        super().__init__()
        self.context_proj = layers.Dense(embed_dim, name="context_projection")
        self.dropout = layers.Dropout(0.1)
        self.ln = layers.LayerNormalization(epsilon=1e-6)

    def call(self, enc_out, context, training=False):
        context_emb = self.context_proj(context)
        context_emb = self.dropout(context_emb, training=training)
        context_emb = tf.expand_dims(context_emb, 1)

        combined = tf.concat([enc_out, context_emb], axis=1)
        return self.ln(combined)

class DecoderBlock(layers.Layer):
    def __init__(self, embed_dim, ff_dim, num_heads):
        super().__init__()
        self.self_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads)
        self.cross_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads)
        self.ff = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dropout(0.1),
            layers.Dense(embed_dim)
        ])
        self.ln1 = layers.LayerNormalization(epsilon=1e-6)
        self.ln2 = layers.LayerNormalization(epsilon=1e-6)
        self.ln3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(0.1)
        self.dropout2 = layers.Dropout(0.1)

    def _causal_mask(self, seq_len, batch_size):
        mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        mask = tf.cast(mask, tf.bool)
        mask = tf.reshape(mask, (1, seq_len, seq_len))
        return tf.repeat(mask, repeats=batch_size, axis=0)

    def call(self, x, enc_out, training=False):
        batch = tf.shape(x)[0]
        seq_len = tf.shape(x)[1]
        mask = self._causal_mask(seq_len, batch)

        att1 = self.self_att(x, x, attention_mask=mask)
        att1 = self.dropout1(att1, training=training)
        x = self.ln1(x + att1)

        att2 = self.cross_att(x, enc_out)
        att2 = self.dropout2(att2, training=training)
        x = self.ln2(x + att2)

        f = self.ff(x, training=training)
        x = self.ln3(x + f)
        return x

def build_vit_encoder():
    img = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    patches = PatchExtract(PATCH_SIZE)(img)
    x = PatchEmbedding(NUM_PATCHES, EMBED_DIM)(patches)

    for _ in range(ENCODER_LAYERS):
        x = EncoderBlock(EMBED_DIM, NUM_HEADS, MLP_DIM)(x)

    return tf.keras.Model(img, x, name="vit_encoder")

def build_decoder():
    seq_in = layers.Input(shape=(MAX_LEN-1,), dtype=tf.int32)
    enc_out = layers.Input(shape=(NUM_PATCHES + 1, EMBED_DIM))

    tok_emb = layers.Embedding(VOCAB_SIZE, EMBED_DIM)(seq_in)
    pos_ids = tf.range(start=0, limit=MAX_LEN-1, delta=1)
    pos_layer = layers.Embedding(MAX_LEN, EMBED_DIM)
    pos_emb = pos_layer(pos_ids)
    pos_emb = tf.expand_dims(pos_emb, 0)
    x = tok_emb + pos_emb

    for _ in range(DECODER_LAYERS):
        x = DecoderBlock(EMBED_DIM, MLP_DIM, NUM_HEADS)(x, enc_out)

    out = layers.Dense(VOCAB_SIZE)(x)
    return tf.keras.Model([seq_in, enc_out], out, name="decoder")

encoder = build_vit_encoder()
decoder = build_decoder()
context_fusion = ContextFusion(EMBED_DIM)

img_inp = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="image_input")
context_inp = layers.Input(shape=(CONTEXT_DIM,), name="context_input")
seq_inp = layers.Input(shape=(MAX_LEN-1,), dtype=tf.int32, name="sequence_input")

enc_out = encoder(img_inp)
fused_out = context_fusion(enc_out, context_inp)
dec_out = decoder([seq_inp, fused_out])

model = tf.keras.Model([img_inp, context_inp, seq_inp], dec_out)

print("✓ Model architecture built")

print("\nLoading trained weights...")
model.load_weights(WEIGHTS_PATH)
print("✓ Weights loaded")

# ============================================================================
# INITIALIZE FEATURE EXTRACTORS
# ============================================================================

feature_extractor = CompleteFeatureExtractor()

print("\n" + "="*80)
print("✓ ALL SYSTEMS READY - 180D MSCOCO FEATURES")
print("="*80)

# ============================================================================
# CAPTION GENERATION FUNCTION
# ============================================================================

def generate_caption(image_path, temperature=1.0, max_length=40, verbose=True):
    """Generate caption using 180D MSCOCO features"""

    if verbose:
        print("\n" + "="*80)
        print(f"GENERATING CAPTION")
        print("="*80)
        print(f"Image: {os.path.basename(image_path)}")

    # Load image
    if verbose:
        print("\n[1/4] Loading image...")

    img_pil = Image.open(image_path).convert('RGB')
    img_array = np.array(img_pil.resize((IMG_SIZE, IMG_SIZE))).astype(np.float32) / 255.0
    img_cv = cv2.imread(image_path)

    # Extract 180D features
    if verbose:
        print("\n[2/4] Extracting 180D MSCOCO features...")
        print("  - 80D: Object detection")
        print("  - 91D: Stuff detection")
        print("  - 9D: Scene statistics")

    context_features, detected_objects = feature_extractor.extract_features(img_cv)

    if verbose:
        print(f"\n✓ Detected {len(detected_objects)} objects:")
        for obj in detected_objects[:5]:
            print(f"    - {obj['name']}: {obj['confidence']:.2f}")
        if len(detected_objects) > 5:
            print(f"    ... and {len(detected_objects)-5} more")

    # Prepare inputs
    img_batch = np.expand_dims(img_array, axis=0)

    # Pad or truncate context to match CONTEXT_DIM
    if len(context_features) != CONTEXT_DIM:
        if len(context_features) < CONTEXT_DIM:
            # Pad with zeros
            padding = np.zeros(CONTEXT_DIM - len(context_features), dtype=np.float32)
            context_features = np.concatenate([context_features, padding])
        else:
            # Truncate
            context_features = context_features[:CONTEXT_DIM]

    context_batch = np.expand_dims(context_features, axis=0)

    # Encode image
    if verbose:
        print("\n[3/4] Encoding image...")

    enc_output = encoder.predict(img_batch, verbose=0)
    fused_features = context_fusion(enc_output, context_batch, training=False)

    # Generate caption
    if verbose:
        print("\n[4/4] Generating caption...")

    start_token = tokenizer.word_index.get('startseq', 1)
    end_token = tokenizer.word_index.get('endseq', 2)

    sequence = [start_token]

    for _ in range(max_length):
        padded = tf.keras.preprocessing.sequence.pad_sequences(
            [sequence], maxlen=MAX_LEN-1, padding='post'
        )

        preds = decoder.predict([padded, fused_features], verbose=0)
        logits = preds[0, len(sequence)-1, :]
        logits = logits / temperature

        probs = tf.nn.softmax(logits).numpy()
        next_word = np.argmax(probs)

        if next_word == end_token or next_word == 0:
            break

        sequence.append(next_word)

    # Convert to caption
    caption = ' '.join([idx_to_word.get(idx, '') for idx in sequence[1:]])
    caption = caption.replace('startseq', '').replace('endseq', '').strip()

    if caption:
        caption = caption[0].upper() + caption[1:]

    if verbose:
        print("\n" + "="*80)
        print("RESULT:")
        print("="*80)
        print(f"Caption: {caption}")
        print("="*80)

    return caption

# ============================================================================
# MAIN USAGE
# ============================================================================

if __name__ == "__main__":
    print("\n" + "="*80)
    print("USAGE")
    print("="*80)
    print("""
# Generate caption:
caption = generate_caption('/content/test_image.jpg')

# With more creativity:
caption = generate_caption('/content/test_image.jpg', temperature=1.2)

# Silent mode:
caption = generate_caption('/content/test_image.jpg', verbose=False)
    """)

    print("\n" + "="*80)
    print("READY FOR INFERENCE!")
    print("="*80)
    print("\nNote: If CONTEXT_DIM != 180, you need to retrain with 180D features")
    print("="*80)

AUTOMATIC IMAGE CAPTIONING - 180D MSCOCO Features
Loading models and dependencies...

[1/5] Loading configuration...
✓ Configuration loaded
  Context dimension: 180D

[2/5] Loading tokenizer...
✓ Tokenizer loaded (8831 words)

[3/5] Loading object detector...

[4/5] Loading stuff detector...

[5/5] Building model architecture...
✓ Model architecture built

Loading trained weights...
✓ Weights loaded
  ✓ Object detector loaded
  ✓ Stuff detector loaded (heuristic-based)

✓ Complete feature extractor initialized (180D)

✓ ALL SYSTEMS READY - 180D MSCOCO FEATURES

USAGE

# Generate caption:
caption = generate_caption('/content/test_image.jpg')

# With more creativity:
caption = generate_caption('/content/test_image.jpg', temperature=1.2)

# Silent mode:
caption = generate_caption('/content/test_image.jpg', verbose=False)
    

READY FOR INFERENCE!

Note: If CONTEXT_DIM != 180, you need to retrain with 180D features


In [ ]:
generate_caption('/content/test_image.jpg')


GENERATING CAPTION
Image: test_image.jpg

[1/4] Loading image...

[2/4] Extracting 180D MSCOCO features...
  - 80D: Object detection
  - 91D: Stuff detection
  - 9D: Scene statistics

✓ Detected 20 objects:
    - person: 0.82
    - person: 0.71
    - person: 0.65
    - person: 0.63
    - person: 0.61
    ... and 15 more

[3/4] Encoding image...

[4/4] Generating caption...

RESULT:
Caption: A child in the ocean


'A child in the ocean'